In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

import datetime
import time

from cmath import nan

data_de_hoje = datetime.date.today()
data_de_hoje = data_de_hoje.strftime('%d/%m/%Y')

import re

In [ ]:
import os
os.chdir("/content/drive/My Drive")
!ls

In [ ]:
## link para o site onde tem alterações que pode fazer nos componentes dos gráficos
#https://plotly.com/python/marker-style/

In [2]:
precos_carrefour = pd.read_excel('precos_lojas.xlsx')


In [3]:
# Baixando Tabela Geral
#precos_carrefour = pd.read_excel('/Volumes/GoogleDrive-102559694260234423878/.shortcut-targets-by-id/1-0LIbarMSRLTFnKiMOFQuwh3KZ880_sp/JubartData/codigo/desenvolvimento/painel-dos-precos/Carrefour Nacional/precos_carrefour_carnes_allcities_2024_02_01.xlsx')

#precos_carrefour = pd.DataFrame(precos_carrefour)
#precos_carrefour = precos_carrefour.drop_duplicates()
#precos_carrefour

df = {
    "produto":[],
    "preco":[],
    "desconto":[],
    "loja":[],
    "cidade/UF":[],
    'estado':[],
    "data": precos_carrefour["data"].tolist()
}

for i in precos_carrefour[0].tolist():
    info = i.split("\n")

    if len(info) == 2:


        produto = info[0]

        preco_loja, cidade_estado = info[1].split(", ")

        cidade, estado = cidade_estado.split(" - ")

        preco_loja = preco_loja.lstrip("R$ ")
        space = preco_loja.find(" ")
        preco = preco_loja[:space]
        loja = preco_loja[space+1:]

        desconto = 0

    elif len(info) == 3:


        produto = info[0]
        preco = info[1].lstrip("R$ ")

        loja, cidade_estado = info[2].split(", ")

        cidade, estado = cidade_estado.split(" - ")

        _, loja = loja.split("/kg ")
        desconto = 0

    elif len(info) == 4:
        desconto_produto = info[0].split("%")

        if len(desconto_produto) == 2:
            desconto = int(desconto_produto[0])
            produto = info[1]
        else:
            desconto = int(info[2][1:].rstrip("%"))
            produto = desconto_produto[0]

        preco_loja, cidade_estado = info[3].split(", ")

        cidade, estado = cidade_estado.split(" - ")

        preco_loja = preco_loja.lstrip("R$ ")
        space = preco_loja.find(" ")
        preco = preco_loja[:space]
        loja = preco_loja[space+1:]

    elif len(info) == 5:

        desconto = int(info[0].split("%")[0])
        produto = info[1]
        preco = info[3].lstrip("R$ ")

        loja, cidade_estado = info[4].split(", ")

        cidade, estado = cidade_estado.split(" - ")

        _, loja = loja.split("/kg ")

    elif len(info) == 6:
        desconto = int(info[4][1:].rstrip("%"))
        produto = info[1]

        preco_loja, cidade_estado = info[5].split(", ")

        cidade, estado = cidade_estado.split(" - ")

        preco_loja = preco_loja.lstrip("R$ ")
        space = preco_loja.find(" ")
        preco = preco_loja[:space]
        loja = preco_loja[space+1:]


    preco = preco.replace(",", ".")
    preco = float(preco)
    df["produto"].append(produto)
    df["preco"].append(preco)
    df["desconto"].append(desconto)
    df["loja"].append(loja)
    df["cidade/UF"].append(cidade+'/'+estado)
    df["estado"].append(estado)


precos_carrefour_prod = pd.DataFrame().from_dict(df)
precos_carrefour_prod = precos_carrefour_prod.drop_duplicates()
precos_carrefour_prod.head(5)


,produto,preco,desconto,loja,cidade/UF,estado,data
0,Carne Moída Acém Carrefour Aprox. 500 g,17.24,0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,21/02/2024
1,Lingüiça de Calabresa Defumada Sadia Aprox. 500g,17.99,0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,21/02/2024
2,Peito de Frango sem Osso Congelado Carrefour A...,25.49,0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,21/02/2024
3,Pedaços de Sobrecoxa de Frango Congelado Sadia...,13.89,0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,21/02/2024
4,"Picanha Bovina Congelada Aprox 1,6Kg",148.78,0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,21/02/2024


In [26]:
### Encontrando os produtos que ocorrem com mais frequencia
lista_lojas = precos_carrefour_prod.groupby(['estado', 'cidade/UF', 'loja']).count()
#lista_loja = lista_lojas.sort_values(by=['estado'], ascending=False)
lista_lojas

produto  \
estado cidade/UF    loja                                                    
AL     Maceió/AL    Hiper Maceió JatiúcaMangabeiras                    17   
AM     Manaus/AM    Hipermercado Manaus AdrianópolisAdrianopolis       20   
                    Hipermercado Manaus ShoppingChapada                20   
                    Hipermercado Ponta NegraDom Pedro                  20   
BA     Itabuna/BA   Big Bompreço ItabunaGoes Calmon                    20   
...                                                                   ...   
SP     São Paulo/SP Hipermercado InterlagosCampo Grande                20   
                    Hipermercado LimãoLimao                            20   
                    Hipermercado PamplonaJd Pauilsta                   19   
                    Hipermercado PinheirosVila Gomes                   20   
                    Hipermercado TietêVila Guilherme                   20   

                                                                  preco  \
estado cidade/UF    loja                                                  
AL     Maceió/AL    Hiper Maceió JatiúcaMangabeiras                  17   
AM     Manaus/AM    Hipermercado Manaus AdrianópolisAdrianopolis     20   
                    Hipermercado Manaus ShoppingChapada              20   
                    Hipermercado Ponta NegraDom Pedro                20   
BA     Itabuna/BA   Big Bompreço ItabunaGoes Calmon                  20   
...                                                                 ...   
SP     São Paulo/SP Hipermercado InterlagosCampo Grande              20   
                    Hipermercado LimãoLimao                          20   
                    Hipermercado PamplonaJd Pauilsta                 19   
                    Hipermercado PinheirosVila Gomes                 20   
                    Hipermercado TietêVila Guilherme                 20   

                                                                  desconto  \
estado cidade/UF    loja                                                     
AL     Maceió/AL    Hiper Maceió JatiúcaMangabeiras                     17   
AM     Manaus/AM    Hipermercado Manaus AdrianópolisAdrianopolis        20   
                    Hipermercado Manaus ShoppingChapada                 20   
                    Hipermercado Ponta NegraDom Pedro                   20   
BA     Itabuna/BA   Big Bompreço ItabunaGoes Calmon                     20   
...                                                                    ...   
SP     São Paulo/SP Hipermercado InterlagosCampo Grande                 20   
                    Hipermercado LimãoLimao                             20   
                    Hipermercado PamplonaJd Pauilsta                    19   
                    Hipermercado PinheirosVila Gomes                    20   
                    Hipermercado TietêVila Guilherme                    20   

                                                                  data  
estado cidade/UF    loja                                                
AL     Maceió/AL    Hiper Maceió JatiúcaMangabeiras                 17  
AM     Manaus/AM    Hipermercado Manaus AdrianópolisAdrianopolis    20  
                    Hipermercado Manaus ShoppingChapada             20  
                    Hipermercado Ponta NegraDom Pedro               20  
BA     Itabuna/BA   Big Bompreço ItabunaGoes Calmon                 20  
...                                                                ...  
SP     São Paulo/SP Hipermercado InterlagosCampo Grande             20  
                    Hipermercado LimãoLimao                         20  
                    Hipermercado PamplonaJd Pauilsta                19  
                    Hipermercado PinheirosVila Gomes                20  
                    Hipermercado TietêVila Guilherme                20  

[117 rows x 4 columns]

In [ ]:
lista_lojas.to_excel('lista_lojas.xlsx')

FEZ UM FILTRO DE UM PRODUTO A PARTIR DE PALAVRAS CHAVES: 
ESSE DATA FRAME PARTIU DA BUSCA NO SITE COM A PALAVRA: 'FILÉ TILÁPIA' MAS PEGA INCLUSIVRE PRECÓS DE FILÉ MIGNON

PODE TER UM DROPDOWN AQUI, TIPO: PRODUTO FRESCO OU CONGELADO

In [10]:
## Selecionando os dados por Palavras-chaves
list_produto = precos_carrefour_prod[precos_carrefour_prod['produto'].str.contains('Picanha', flags=re.IGNORECASE, na=False, regex=True)]
#list_produto = list_produto[list_produto['produto'].str.contains('125', flags=re.IGNORECASE, na=False, regex=True)]
#list_produto = list_produto[list_produto['produto'].str.contains('Congelado', flags=re.IGNORECASE, na=False, regex=True)]
list_produto['produto'].unique()

array(['Picanha Bovina Congelada Aprox 1,6Kg',
       'Picanha Resfriada Maturatta Aprox. 1,2Kg',
       'Picanha Bovina Peça Carrefour Aproximadamente 800 g'],
      dtype=object)

In [11]:
list_produto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 4 to 2379
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   produto    185 non-null    object 
 1   preco      185 non-null    float64
 2   desconto   185 non-null    int64  
 3   loja       185 non-null    object 
 4   cidade/UF  185 non-null    object 
 5   estado     185 non-null    object 
 6   data       185 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 11.6+ KB


Frequencia dos Produtos

In [12]:
### Encontrando os produtos que ocorrem com mais frequencia
occur = list_produto.groupby(['produto']).count()
c = occur.sort_values(by=['data'], ascending=False)
c.head(10)

,preco,desconto,loja,cidade/UF,estado,data
produto,,,,,,
"Picanha Bovina Congelada Aprox 1,6Kg",97,97,97,97,97,97
"Picanha Resfriada Maturatta Aprox. 1,2Kg",87,87,87,87,87,87
Picanha Bovina Peça Carrefour Aproximadamente 800 g,1,1,1,1,1,1


In [13]:
## Criando uma lista com os 10 produtos mais frequentes
top_10 = list(c.head(10).index.unique())
top_10

['Picanha Bovina Congelada Aprox 1,6Kg',
 'Picanha Resfriada Maturatta Aprox. 1,2Kg',
 'Picanha Bovina Peça Carrefour Aproximadamente 800 g']

In [14]:
#### Reduzindo o dataframe a apenas os 10 itens mais frequentes
list_produto = list_produto.loc[list_produto['produto'].isin(top_10)]

Numero de produtos ofertados por Cidade

In [15]:
list_produto.columns.unique()

Index(['produto', 'preco', 'desconto', 'loja', 'cidade/UF', 'estado', 'data'], dtype='object')

In [16]:
### Encontrando os produtos que ocorrem com mais frequencia
occur = list_produto.groupby(['cidade/UF']).count().reset_index()
d = occur.sort_values(by=['data'], ascending=False).head(60)
d

,cidade/UF,produto,preco,desconto,loja,estado,data
56,São Paulo/SP,24,24,24,24,24,24
6,Campinas/SP,9,9,9,9,9,9
4,Brasília/DF,8,8,8,8,8,8
42,Recife/PE,7,7,7,7,7,7
44,Rio de Janeiro/RJ,7,7,7,7,7,7
50,São Bernardo do Campo/SP,7,7,7,7,7,7
30,Manaus/AM,6,6,6,6,6,6
3,Belo Horizonte/MG,6,6,6,6,6,6
43,Ribeirão Preto/SP,6,6,6,6,6,6
48,Santo André/SP,6,6,6,6,6,6


In [18]:
### Encontrando os produtos que ocorrem com mais frequencia
occur = list_produto.groupby(['loja', 'cidade/UF']).count().reset_index()
d = occur.sort_values(by=['data'], ascending=False).head(60)
d

,loja,cidade/UF,produto,preco,desconto,estado,data
95,Hipermercado Santa MariaCentro,Santa Maria/RS,3,3,3,3,3
56,Hipermercado DiademaCentro,Diadema/SP,2,2,2,2,2
80,Hipermercado Norte ShoppingCachambi,Rio de Janeiro/RJ,2,2,2,2,2
77,Hipermercado ManilhaNeves,São Gonçalo/RJ,2,2,2,2,2
76,Hipermercado Manaus ShoppingChapada,Manaus/AM,2,2,2,2,2
75,Hipermercado Manaus AdrianópolisAdrianopolis,Manaus/AM,2,2,2,2,2
73,Hipermercado LimãoLimao,São Paulo/SP,2,2,2,2,2
71,Hipermercado Juiz de ForaGraminha,Juiz de Fora/MG,2,2,2,2,2
69,Hipermercado InterlagosCampo Grande,São Paulo/SP,2,2,2,2,2
68,Hipermercado ImigrantesBosque da Saúde,São Paulo/SP,2,2,2,2,2


In [19]:
len(list_produto['produto'].unique())

3

In [20]:
len(list_produto['cidade/UF'].unique())

61

In [22]:
fig = px.scatter(list_produto, x="loja", y='preco', color='produto',  
                 hover_data=list_produto.columns.unique(), 
                 title="Preços Filés de Tilápia Congelados por cidade",
                 width=1200, height=600)

fig.update_traces(marker=dict(size=6, 
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers')
                  )

fig.show()

AQUI A GENTE FAZ UMA SELEÇÃO DE ALGUNS PRODUTOS PARA COMPARAR

AQUI pode ir um DROPDOWN com os produtos mais frequentes

In [ ]:
['Sardinhas Ao Molho De Tomate Picante General 125g',
 'Sardinhas Em Óleo Comestivel General 125g',
 'Sardinhas Ao Molho De Tomate General 125g',
 'Sardinhas Em Azeite De Oliva General 125g',
 'Sardinhas Ao Natural General 125g',
 'Sardinha com Óleo Coqueiro 125g',
 'Sardinha em Óleo Gomes da Costa Sabor Limão 125g',
 'Sardinha com Molho de Tomate Gomes da Costa 125g',
 'Sardinha em Óleo Gomes da Costa 125g',
 'Sardinha em Óleo 88 125g']

In [14]:
disp_preco_produto = list_produto.loc[(list_produto['produto']=='Sardinha em Óleo Gomes da Costa 125g') |
                                                (list_produto['produto']=='Sardinha com Óleo Coqueiro 125g') |
                                                (list_produto['produto']=='Sardinhas Em Óleo Comestivel General 125g') |
                                                (list_produto['produto']=='Sardinha em Óleo 88 125g')
                                               ]
#disp_preco_produto

fig = px.scatter(disp_preco_produto, x="cidade/UF", y='preco', 
                color='produto',  hover_data=disp_preco_produto.columns.unique(), 
                title = "Preços de Filé de Tilápia Congelado por marca",
                width=1200, height=600)

fig.update_traces(marker=dict(size=6,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

fig_2 = px.box(disp_preco_produto, x="estado", y='preco', color='produto',  hover_data=disp_preco_produto.columns.unique(), title = "Distribuição de Preços de Filé de Tilápia Congelado por marca")
fig_2.show()

In [26]:
import plotly.io as pio

pio.write_image(fig, 'Preços de Filé de Tilápia Congelado por marca.png', width=1500, height=700)
#pio.write_image(fig_2, 'Distribuição de Preços de Filé de Tilápia Congelado por marca.png', width=1500, height=700)

SLECAO POR ESTADO

In [18]:
list_produto['estado'].unique()

array(['AL', 'BA', 'AM', 'DF', 'ES', 'GO', 'MG', 'MS', 'PB', 'PE', 'PR',
       'RJ', 'RN', 'RS', 'SC', 'SP'], dtype=object)

In [19]:
#selecao produto 1

selecao_estado = input ('Escolha palavra chave para produto: - separar palavras chaves por | ')

#dist_uf = list_produto.loc[list_produto['estado']==selecao_estado]
dist_uf = list_produto.loc[list_produto['estado']==selecao_estado]

dist_uf = dist_uf.sort_values(by= 'preco')

fig = px.scatter(dist_uf, x="cidade/UF", y='preco', color='produto',  
                 hover_data=dist_uf.columns.unique(),
                 title = "Preços de Filé de Tilápia Congelado por marca no estado de " + selecao_estado,
                 width=1200, height=600)

fig.update_traces(marker=dict(size=6,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [20]:
fig_2 = px.box(dist_uf, x="cidade/UF", y='preco', color='cidade/UF',  
               hover_data=disp_preco_produto.columns.unique(), 
               title = "Distribuição de Preços de Filé de Tilápia Congelado no estado de " + selecao_estado)
fig_2.show()

In [13]:
dist_uf.columns.unique()

Index(['produto', 'preco', 'desconto', 'loja', 'cidade/UF', 'estado', 'data'], dtype='object')

In [23]:
dist_uf.produto.unique()

array(['Sardinha em Óleo 88 125g', 'Sardinhas Ao Natural General 125g',
       'Sardinha com Óleo Coqueiro 125g',
       'Sardinha com Molho de Tomate Gomes da Costa 125g',
       'Sardinha em Óleo Gomes da Costa Sabor Limão 125g',
       'Sardinha em Óleo Gomes da Costa 125g',
       'Sardinhas Em Óleo Comestivel General 125g',
       'Sardinhas Em Azeite De Oliva General 125g',
       'Sardinhas Ao Molho De Tomate General 125g',
       'Sardinhas Ao Molho De Tomate Picante General 125g'], dtype=object)

In [21]:
#### Criando PESO CONVERTIDO PARA KG
dist_uf['peso'] = dist_uf['produto'].str[-4:-1]
dist_uf['peso'] = dist_uf['peso'].astype(float)

dist_uf['preco_convertido_kg'] = (dist_uf['preco'] / dist_uf['peso'])*1000


dist_uf = dist_uf.sort_values(by= 'preco_convertido_kg')

fig = px.scatter(dist_uf, x="cidade/UF", y='preco_convertido_kg', color='produto',  
                 hover_data=dist_uf.columns.unique(),
                 title = "Distribuição de Preços Convertidos para Kg de Filé de Tilápia Congelado no estado de " + selecao_estado,
                 width=1200, height=600)

fig.update_traces(marker=dict(size=6,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

SELECIONE UMA CIDADE

LEO - Aqui vai um DROPDOWN POR CIDADE

In [25]:
list_produto['cidade/UF'].unique()

array(['Maceió/AL', 'Itabuna/BA', 'Manaus/AM', 'Brasília/DF',
       'Taguatinga/DF', 'Vila Velha/ES', 'Anapolis/GO', 'Goiania/GO',
       'Belo Horizonte/MG', 'Contagem/MG', 'Juiz de Fora/MG',
       'Uberlandia/MG', 'Campo Grande/MS', 'Campina Grande/PB',
       'João Pessoa/PB', 'Jaboatão dos Guararapes/PE', 'Petrolina/PE',
       'Recife/PE', 'Curitiba/PR', 'Londrina/PR', 'Pinhais/PR',
       'Belford Roxo/RJ', 'Duque de Caxias/RJ', 'Rio de Janeiro/RJ',
       'São Gonçalo/RJ', 'Campos dos Goytacazes/RJ', 'Macaé/RJ',
       'Mossoró/RN', 'Natal/RN', 'Canoas/RS', 'Caxias do Sul/RS',
       'Gravataí/RS', 'Novo Hamburgo/RS', 'Pelotas/RS', 'Santa Maria/RS',
       'Viamão/RS', 'Balneário Camboriú/SC', 'Florianópolis/SC',
       'Barueri/SP', 'Caçapava/SP', 'Campinas/SP', 'Diadema/SP',
       'Guarujá/SP', 'Guarulhos/SP', 'Indaiatuba/SP', 'Itu/SP',
       'Jundiai/SP', 'Piracicaba/SP', 'Praia Grande/SP',
       'Presidente Prudente/SP', 'Ribeirão Preto/SP', 'Salto/SP',
       'Santana 

In [26]:
#selecao produto 1
list_produto['cidade/UF'].unique()
selecao_cidade = input ('Escolha palavra chave para produto: - separar palavras chaves por | ')

dist_cidade_uf = list_produto.loc[list_produto['cidade/UF']==selecao_cidade]
dist_cidade_uf = dist_cidade_uf.sort_values(by= 'preco')

fig = px.scatter(dist_cidade_uf, x="produto", y='preco', color='produto',  hover_data=dist_cidade_uf.columns.unique(), title= "Produtos em "+selecao_cidade )
fig.show()